In [1]:
import copy
import json
import numpy as np
import pandas as pd
import os
import requests
import time
from pathlib import Path

import PySAM.Battery as battery_model
import PySAM.Pvsamv1 as pv_model
import PySAM.Utilityrate5 as utility_rate
import PySAM.Cashloan as cashloan
import PySAM.ResourceTools
import PySAM.UtilityRateTools

file_dir = os.path.abspath('')

In [2]:
def get_urdb_rate_data(page, key):
    urdb_url = 'https://api.openei.org/utility_rates?format=json&detail=full&version=8'
    get_url = urdb_url + '&api_key={api_key}&getpage={page_id}'.format(api_key=key, page_id=page)

    filename = "urdb_rate_{}.json".format(page)
    print(filename)

    if not os.path.isfile(filename):
        print(get_url)
        resp = requests.get(get_url, verify=False)
        data = resp.text
        with open(filename, 'w') as f:
            f.write(json.dumps(data, sort_keys=True, indent=2, separators=(',', ': ')))
    else:
        with open(filename, 'r') as f:
            data = json.load(f)

    return data

def get_load_profile(load_path):
    df = pd.read_csv(load_path, dtype=float)
    return pd.to_numeric(df.iloc[:, 0]).values

In [3]:
def poller(url: str, poll_interval: int = 2):
    """
    Function for polling the REopt API results URL until status is not "Optimizing..."

    :param url: Results url to poll
    :type url: str

    :param poll_interval: poll interval time in seconds
    :type url: float

    :return: The dictionary response from the API (once status is not "Optimizing...")
    :rtype: dict

    """
    key_error_count = 0
    key_error_threshold = 4
    status = "Optimizing..."
    while True:
        resp = requests.get(url=url, verify=False)
        resp_dict = json.loads(resp.text)

        try:
            status = resp_dict['outputs']['Scenario']['status']
        except KeyError:
            key_error_count += 1
            if key_error_count > key_error_threshold:
                break

        if status != "Optimizing...":
            break
        else:
            time.sleep(poll_interval)
    return resp_dict

In [4]:
def get_battery_schedule(reopt_json):
    pv_to_battery = reopt_json['outputs']['Scenario']['Site']['PV']['year_one_to_battery_series_kw']
    grid_to_battery = reopt_json['outputs']['Scenario']['Site']['ElectricTariff']['year_one_to_battery_series_kw']
    battery_to_load = reopt_json['outputs']['Scenario']['Site']['Storage']['year_one_to_load_series_kw']
    battery_to_grid = reopt_json['outputs']['Scenario']['Site']['Storage']['year_one_to_grid_series_kw']

    battery_schedule = []

    for i in range(len(pv_to_battery)):
        charge = (pv_to_battery[i] + grid_to_battery[i])
        discharge = battery_to_load[i] + battery_to_grid[i]
        battery_schedule.append(discharge - charge)

    return battery_schedule

In [6]:
def call_reopt_or_load_results(urdb_response, lat, lon, load_profile, pv, key, sized_already, params_dict):
    """
    Call the REopt API

    :return: a dictionary of REopt results
    :rtype: dict
    """
    results = dict()

    reopt_api_post_url = 'https://developer.nrel.gov/api/reopt/v1/job?format=json'
    reopt_api_poll_url = 'https://developer.nrel.gov/api/reopt/v1/job/'

    post = {}
    post["Scenario"] = {}
    post["Scenario"]["Site"] = {}
    post["Scenario"]["Site"]["Generator"] = {}
    post["Scenario"]["Site"]["ElectricTariff"] = {}
    post["Scenario"]["Site"]["LoadProfile"] = {}
    post["Scenario"]["Site"]["PV"] = {}
    post["Scenario"]["Site"]["Storage"] = {}

    gen_year_1 = np.array(pv.Outputs.gen[0:8760])  # Ensure battery is off for this
    gen_year_1 = np.clip(gen_year_1, 0, max(gen_year_1))  # Remove inverter night time losses for REopt
    capacity = pv.SystemDesign.system_capacity
    ac_dc_ratio = 1.2
    post["Scenario"]["Site"]["PV"]["dc_ac_ratio"] = ac_dc_ratio  # TODO - find a way to vary this system by system

    print("PV Capacity " + str(capacity))

    # TODO - consider re-modifying this
    post["Scenario"]["Site"]["ElectricTariff"]["net_metering_limit_kw"] = 5000  # https://bvirtualogp.pr.gov/ogp/Bvirtual/leyesreferencia/PDF/2-ingles/114-2007.pdf - assuming primary voltage for the utility rate

    if sized_already:
        ac_capacity = capacity / ac_dc_ratio
        post["Scenario"]["Site"]["PV"]["prod_factor_series_kw"] = list(gen_year_1 / ac_capacity)
        post["Scenario"]["Site"]["PV"]["inv_eff"] = 0.995
        post["Scenario"]["Site"]["PV"]["losses"] = 0.0

        # Fix the PV size on the third iteration
        if True:
            post["Scenario"]["Site"]["PV"]["max_kw"] = ac_capacity
            post["Scenario"]["Site"]["PV"]["min_kw"] = ac_capacity
        if False:
            # Use the exiting batt size for dispatch:
            batt_kw = pv.BatterySystem.batt_power_discharge_max_kwac
            batt_kwh = pv.BatterySystem.batt_computed_bank_capacity * 0.96 # DC to AC efficiency

            post["Scenario"]["Site"]["Storage"]["max_kw"] = batt_kw
            post["Scenario"]["Site"]["Storage"]["min_kw"] = batt_kw
            post["Scenario"]["Site"]["Storage"]["max_kwh"] = batt_kwh
            post["Scenario"]["Site"]["Storage"]["min_kwh"] = batt_kwh

    # Setup utility rate
    post["Scenario"]["Site"]["ElectricTariff"]["urdb_response"] = urdb_response

    post['Scenario']['Site']["latitude"] = lat
    post['Scenario']['Site']["longitude"] = lon

    post["Scenario"]["Site"]["LoadProfile"]["loads_kw"] = list(load_profile)
    post["Scenario"]["Site"]["LoadProfile"]["critical_load_pct"] = 0.5

    post["Scenario"]["Site"]["Storage"]["internal_efficiency_pct"] = 0.9818

    post["Scenario"]["Site"]["PV"]["can_curtail"] = True
    post["Scenario"]["Site"]["PV"]["gcr"] = 0.3
    post["Scenario"]["Site"]["PV"]["degradation_pct"] = 0.0

    # Commercial defaults from SAM 2022.11.21
    # These are rolled up into total installed cost and not available in the pysam json
    post["Scenario"]["Site"]["PV"]["installed_cost_us_dollars_per_kw"] = 1940
    post["Scenario"]["Site"]["Storage"]["installed_cost_us_dollars_per_kw"] = 405.56
    post["Scenario"]["Site"]["Storage"]["installed_cost_us_dollars_per_kwh"] = 225.06
    post["Scenario"]["Site"]["Storage"]["canGridCharge"] = params_dict["grid_charging"]
    post["Scenario"]["Site"]["Storage"]["soc_min_fraction"] = 0.7

    if True:
        post["Scenario"]["Site"]["LoadProfile"]["outage_start_time_step"] = 4543 # 6 am July 9th
        post["Scenario"]["Site"]["LoadProfile"]["outage_end_time_step"] = 4569 # 6 am July 10th
        post["Scenario"]["Site"]["Generator"]["max_kw"] = 0

    filename = "reopt_results" + os.sep + "reopt_results_outage_{}_{}_{}_{}.json".format(params_dict["actual"], params_dict["grid_charging"], lat, lon)
    print(filename)
    if not os.path.isfile(filename):
        post_url = reopt_api_post_url + '&api_key={api_key}'.format(api_key=key)
        # print(post)
        resp = requests.post(post_url, json.dumps(post), verify=False)

        if resp.ok:

            run_id_dict = json.loads(resp.text)
            try:
                run_id = run_id_dict['run_uuid']
            except KeyError:
                msg = "Response from {} did not contain run_uuid.".format(post_url)
                raise KeyError(msg)

            poll_url = reopt_api_poll_url + '{run_uuid}/results/?api_key={api_key}'.format(
                run_uuid=run_id,
                api_key=key)
            reopt_json = poller(url=poll_url)

            batt_schedule = get_battery_schedule(reopt_json)
            with open(filename, 'w') as f:
                f.write(json.dumps(reopt_json , sort_keys=True, indent=2, separators=(',', ': ')))
        else:
            try:
                text = json.loads(resp.text)
                if "messages" in text.keys():
                    raise Exception(text["messages"])
            except:
                raise Exception(resp.text)
            resp.raise_for_status()

    else:
        with open(filename, 'r') as f:
            reopt_json = json.load(f)
            batt_schedule = reopt_json["batt_schedule"]

    return batt_schedule


In [7]:
def update_total_installed_cost(pv, batt, cl, dispatch_option):
    pv_cost = 1940  # $/kW DC
    batt_kw_cost = 405.56 # $/kW DC
    batt_kwh_cost = 225.06  # $/kWh DC

    pv_size = pv.SystemDesign.system_capacity
    batt_kw = batt.BatterySystem.batt_power_discharge_max_kwac
    batt_kwh = batt.BatterySystem.batt_computed_bank_capacity

    total_cost = pv_size * pv_cost
    if dispatch_option != 6:
        total_cost += batt_kw * batt_kw_cost + batt_kwh_cost * batt_kwh
    cl.value("total_installed_cost", total_cost)

In [8]:
# SAM default bank voltage is 500 V
def run_sam_battery(batt, custom_dispatch, load_profile, dispatch_choice):
    batt.value("en_batt", 1)  # Ensure battery is enabled for this step
    batt.value("batt_ac_or_dc", 1)  # AC
    batt.value("batt_maximum_SOC", 100)  # ReOpt goes to 100%
    batt.value("load", load_profile)

    batt.value("batt_dispatch_choice", dispatch_choice)
    batt.value("batt_dispatch_auto_can_gridcharge", 0)
    batt.value("batt_dispatch_auto_can_charge", 1)
    if dispatch_choice == 3:
        batt.value("batt_custom_dispatch", custom_dispatch)

    # Old code before we had SAM pv gen
    # gen = reopt_json["outputs"]["Scenario"]["Site"]["PV"]["year_one_power_production_series_kw"]
    # batt.value("gen", gen)

    batt.execute(1)

    return batt

In [9]:
def get_pv_json(json_file_path):
    with open(json_file_path) as f:
        dic = json.load(f)
    return dic


In [12]:
key = "RkdVzEGa7mlEGqV9jzwjhrh95ACZOr6PFr0K0yTc"

load_path = file_dir + "/weather_and_load/" + "san_juan_hospital_actual_load.csv"
weather_file = file_dir + "/weather_and_load/" + "18.389862_-66.09338_18.3898_-66.0936_psm3_60_2018.csv"
#pv_path = file_dir + "/size_system_pv_only/" + "pr_hospital_actual_pv_only_pvsamv1.json"
#cashloan_path = file_dir + "/size_system_pv_only/" + "pr_hospital_actual_pv_only_cashloan.json"
pv_path = file_dir + "/size_system_outage/" + "pr_hospital_actual_pv_peak_outage_pvsamv1.json"
cashloan_path = file_dir + "/size_system_outage/" + "pr_hospital_actual_pv_peak_outage_cashloan.json"

lat = 18.389
lon = -66.0933

pv_setup = get_pv_json(pv_path)
load_profile = get_load_profile(load_path)
cashloan_setup = get_pv_json(cashloan_path)

# Create compute modules from imported data
pv = pv_model.default("PVBatteryCommercial")
batt = battery_model.from_existing(pv)
batt.BatteryCell.batt_life_model = 1
ur = utility_rate.from_existing(batt, "PVBatteryCommercial")
cl = cashloan.from_existing(ur, "PVBatteryCommercial")


for k, v in pv_setup.items():
    try:
        pv.value(k, v)
    except AttributeError:
        print("Failed to assign PV key " + str(k))

for k, v in cashloan_setup.items():
    try:
        cl.value(k, v)
    except AttributeError:
        print("Failed to assign cashloan key " + str(k))


dispatch_option = 3

pv.value("analysis_period", 25)
pv.value("solar_resource_file", str(weather_file))
pv.value("dc_degradation", [0.5])
pv.value("en_batt", 0)  # Turn off battery for initial run

# Update utility rate to match json results

page = "5bfdc7925457a33744146c53"
urdb_response = get_urdb_rate_data(page, key)
urdb_response_json = json.loads(urdb_response)["items"][0]
urdb_response_for_sam = copy.deepcopy(urdb_response_json)
rates = PySAM.UtilityRateTools.URDBv8_to_ElectricityRates(urdb_response_for_sam)
for k, v in rates.items():
    try:
        batt.value(k, v)
    except AttributeError:
        ur.value(k, v)

pv.execute(0)

params_dict = {
    "actual" : True,
    "grid_charging" : False
}

sized_already = True

if dispatch_option != 6:
    custom_dispatch = None
    if (dispatch_option == 3):
        custom_dispatch = call_reopt_or_load_results(urdb_response_json, lat, lon, load_profile, pv, key, sized_already, params_dict)

    batt = run_sam_battery(batt, custom_dispatch, load_profile, dispatch_option)
    print(batt.Outputs.average_battery_roundtrip_efficiency)

# Mostly needed for PV-only, but run it each time for consistency
update_total_installed_cost(pv, batt, cl, dispatch_option)

ur.execute(0)

cl.execute(0)

urdb_rate_5bfdc7925457a33744146c53.json
PV Capacity 4399.153416
reopt_results\reopt_results_outage_True_False_18.389_-66.0933.json


c:\Users\bmirletz\Anaconda3\envs\pysam_pip\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\bmirletz\Anaconda3\envs\pysam_pip\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\bmirletz\Anaconda3\envs\pysam_pip\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.w

91.8757890623665


In [15]:
import csv
filename = "reopt_results/reopt_results_outage_True_False_18.389_-66.0933.json"
with open(filename, 'r') as f:
    reopt_json = json.load(f)
    schedule = get_battery_schedule(reopt_json)

    with open("api_schedule.csv", 'w', newline='') as out:
        writer = csv.writer(out, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Battery dispatch"])

        for p in schedule:
            writer.writerow([p])

PV Only:
Run 1 (TMY) - pv size 4744.9719
        "size_kw": 284.70652218768385,
        "size_kwh": 414.1533796079923,

Run 2 (2018 weather, SAM losses) - 
PV: "size_kw": 3719.5067,
        "Storage": {
          "size_kw": 144.24181673755686,
          "size_kwh": 252.8764264631517,

Run 3 (fix PV size):
        "Storage": {
          "size_kw": 138.5428139901768,
          "size_kwh": 242.88215040000003,

Grid charging:
 Run 1: same as above
 Run 2: PV 3627.3012
         "Storage": {
          "size_kw": 151.3433098189313,
          "size_kwh": 280.81047467249095,
Run 3:
          "size_kw": 151.33692170306063,
          "size_kwh": 280.7982464148981,

Outage:
        PV  "size_kw": 3,662.013,
        Storage  "size_kw": 514.463236288942,
          "size_kwh": 4883.736573283864,
          

Outage, 0.7 min soc:
        PV  "size_kw": 3,662.013,
        "Storage": {
          "size_kw": 506.32896364863836,
          "size_kwh": 6473.4246084625665,